<a href="https://colab.research.google.com/github/aaaven/aiarts2020online/blob/master/week11/train_CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install

In [3]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2337, done.
remote: Total 2337 (delta 0), reused 0 (delta 0), pack-reused 2337
Receiving objects: 100% (2337/2337), 8.09 MiB | 26.73 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.


In [4]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 686kB 6.4MB/s 
     |████████████████████████████████| 204kB 14.8MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655251 sha256=cc3cde0804f7a82e710de8ba8d474e39e986ed57f7682ed8784f054e143f6cac
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=bd2fd47938f67b3e2c2bbf2ea193ecf40f5a06c451edab27e58fb57efbe52b84
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

In [6]:
# !bash ./datasets/download_cyclegan_dataset.sh horse2zebra

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple,summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [7]:
# !bash ./scripts/download_cyclegan_model.sh horse2zebra

# If Train Your Own Model
Skip the previous two steps but use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder and name it eg. `sketch2shanshui` as your dataset ad upload it to google drive and get the "id"
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog, sketch2shanshui) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [8]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1lEDFK757PKpcc6mWcDqh9a7x3J9qCiOn

Downloading...
From: https://drive.google.com/uc?id=1lEDFK757PKpcc6mWcDqh9a7x3J9qCiOn
To: /content/pytorch-CycleGAN-and-pix2pix/sketch2shanshui.zip
211MB [00:06, 34.5MB/s]


In [9]:
!unzip 'sketch2shanshui.zip'

Streaming output truncated to the last 5000 lines.
  inflating: sketch2shanshui/trainB/4_of_5_in_086-a.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._4_of_5_in_086-a.jpg  
  inflating: sketch2shanshui/trainB/2_of_5_in_03-a.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._2_of_5_in_03-a.jpg  
  inflating: sketch2shanshui/trainB/0_of_3_in_SS1164-b.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._0_of_3_in_SS1164-b.jpg  
  inflating: sketch2shanshui/trainB/2_of_3_in_SS0393-a.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._2_of_3_in_SS0393-a.jpg  
  inflating: sketch2shanshui/trainB/4_of_9_in_SS1212-b.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._4_of_9_in_SS1212-b.jpg  
  inflating: sketch2shanshui/trainB/1_of_3_in_082-a.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._1_of_3_in_082-a.jpg  
  inflating: sketch2shanshui/trainB/14_of_23_in_06.jpg  
  inflating: __MACOSX/sketch2shanshui/trainB/._14_of_23_in_06.jpg  
  inflating: sketch2shanshui/trainB/1_of_4_in_

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [ ]:
!python train.py --dataroot ./sketch2shanshui --name sketch2shanshui --model cycle_gan --checkpoints_dir /content/drive/MyDrive/0-Colab-Assets/train_cycleGAN --gpu_ids 0

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/0-Colab-Assets/train_cycleGAN	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./sketch2shanshui             	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot sketch2shanshui/trainA --name /content/drive/MyDrive/0-Colab-Assets/train_cycleGAN/sketch2shanshui --model test --no_dropout 